## Importing libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import gc, math

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

/Users/praveengovindaraj/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def mean_without_overflow_fast(col):
    col /= len(col)
    return col.mean() * len(col)

In [4]:
def degToCompass(num):
    val=int((num/22.5)+.5)
    arr=[i for i in range(0,16)]
    return arr[(val % 16)]

## Loading data

In [5]:
%%time
metadata_df = pd.read_csv('data/building_metadata.csv')
train_df = pd.read_csv('data/train.csv', parse_dates=['timestamp'])
test_df = pd.read_csv('data/test.csv', parse_dates=['timestamp'])
weather_train_df = pd.read_csv('data/weather_train.csv', parse_dates=['timestamp'])
weather_test_df = pd.read_csv('data/weather_test.csv', parse_dates=['timestamp'])

CPU times: user 47.8 s, sys: 3.21 s, total: 51 s
Wall time: 51.4 s


## Align timestamps

Timestap data is not in their local time. As energy consumptions are related to the local time, an alighment is nescessary before using timestamp.

The credit goes to this https://www.kaggle.com/nz0722/aligned-timestamp-lgbm-by-meter-type  for the idea. Refer it for more details and explanation about below code.

In [6]:
weather = pd.concat([weather_train_df,weather_test_df],ignore_index=True)
weather_key = ['site_id', 'timestamp']

temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

# calculate ranks of hourly temperatures within date/site_id chunks

temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')

# create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

# Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
site_ids_offsets.index.name = 'site_id'

def timestamp_align(df):
    df['offset'] = df.site_id.map(site_ids_offsets)
    df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
    df['timestamp'] = df['timestamp_aligned']
    del df['timestamp_aligned']
    return df

In [7]:
weather_train_df = timestamp_align(weather_train_df)
weather_test_df = timestamp_align(weather_test_df)

del weather 
del df_2d
del temp_skeleton
del site_ids_offsets

In [8]:
train_df['meter_reading'] = np.log1p(train_df['meter_reading'])
le = LabelEncoder()
metadata_df['primary_use'] = le.fit_transform(metadata_df['primary_use'])

In [9]:
metadata_df = reduce_mem_usage(metadata_df)
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)
weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)

Mem. usage decreased to  0.02 Mb (74.9% reduction)
Mem. usage decreased to 250.63 Mb (59.4% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.20 Mb (70.0% reduction)
Mem. usage decreased to  6.35 Mb (70.0% reduction)


In [10]:
print (f'Training data shape: {train_df.shape}')
print (f'Weather training shape: {weather_train_df.shape}')
print (f'Weather training shape: {weather_test_df.shape}')
print (f'Weather testing shape: {metadata_df.shape}')
print (f'Test data shape: {test_df.shape}')

Training data shape: (20216100, 4)
Weather training shape: (139773, 10)
Weather training shape: (277243, 10)
Weather testing shape: (1449, 6)
Test data shape: (41697600, 4)


In [11]:
%%time
full_train_df = train_df.merge(metadata_df, on='building_id', how='left')
full_train_df = full_train_df.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')

CPU times: user 6.19 s, sys: 2.9 s, total: 9.09 s
Wall time: 9.23 s


In [12]:
# Delete unnecessary dataframes to decrease memory usage
del train_df
del weather_train_df
gc.collect()

156

In [13]:
full_train_df.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'offset'],
      dtype='object')

In [14]:
%%time
# full_test_df = test_df.merge(metadata_df, on='building_id', how='left')
# full_test_df = full_test_df.merge(weather_test_df, on=['site_id', 'timestamp'], how='left')
full_test_df = test_df.merge(metadata_df, on='building_id', how='left')
full_test_df = full_test_df.merge(weather_test_df, on=['site_id', 'timestamp'], how='left')

CPU times: user 12.7 s, sys: 6.31 s, total: 19 s
Wall time: 19.4 s


In [15]:
le = LabelEncoder()
full_train_df['primary_use'] = le.fit_transform(full_train_df['primary_use']).astype(np.int8)
full_test_df['primary_use'] = le.fit_transform(full_test_df['primary_use']).astype(np.int8)

full_train_df['square_feet'] = np.log(full_train_df['square_feet'])
full_test_df['square_feet'] = np.log(full_test_df['square_feet'])
full_train_df['age'] = full_train_df['year_built'].max() - full_train_df['year_built'] + 1
full_test_df['age'] = full_test_df['year_built'].max() - full_test_df['year_built'] + 1

In [16]:
pd.DataFrame(full_train_df.isna().sum().sort_values(ascending=False), columns=['NaN Count'])

,NaN Count
floor_count,16709167
year_built,12127645
age,12127645
cloud_coverage,8831471
precip_depth_1_hr,3758784
wind_direction,1461546
sea_level_pressure,1242486
wind_speed,156584
dew_temperature,113100
air_temperature,109618


In [17]:
missing_values = (100-full_train_df.count() / len(full_train_df) * 100).sort_values(ascending=False)

In [18]:
%%time
missing_features = full_train_df.loc[:, missing_values > 0.0]
missing_features = missing_features.apply(mean_without_overflow_fast)

CPU times: user 3.64 s, sys: 4.2 s, total: 7.84 s
Wall time: 3.52 s


In [19]:
for key in full_train_df.loc[:, missing_values > 0.0].keys():
    if key == 'year_built' or key == 'floor_count':
        full_train_df[key].fillna(math.floor(missing_features[key]), inplace=True)
        full_test_df[key].fillna(math.floor(missing_features[key]), inplace=True)
    else:
        full_train_df[key].fillna(missing_features[key], inplace=True)
        full_test_df[key].fillna(missing_features[key], inplace=True)

In [20]:
def transform(df):
    df['hour'] = np.uint8(df['timestamp'].dt.hour)
    df['day'] = np.uint8(df['timestamp'].dt.day)
    df['weekday'] = np.uint8(df['timestamp'].dt.weekday)
    df['month'] = np.uint8(df['timestamp'].dt.month)
    df['year'] = np.uint8(df['timestamp'].dt.year-1900)
    df['square_feet_floor']=df['square_feet']/df['floor_count']
    df['square_feet_floor']=np.log(df['square_feet_floor'])
    df['square_feet'] = np.log(df['square_feet'])
    return df

In [21]:
def add_lag_feature(weather_df, window=3):
    group_df = weather_df.groupby('site_id')
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
        weather_df[f'{col}_max_lag{window}'] = lag_max[col]
        weather_df[f'{col}_min_lag{window}'] = lag_min[col]
        weather_df[f'{col}_std_lag{window}'] = lag_std[col]
    return None

In [22]:
full_train_df = transform(full_train_df)
full_test_df = transform(full_test_df)

In [23]:
add_lag_feature(full_train_df, window=3)
add_lag_feature(full_train_df, window=72)

add_lag_feature(full_test_df, window=3)
add_lag_feature(full_test_df, window=72)

In [24]:
dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
full_train_df['is_holiday'] = (full_train_df['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
full_test_df['is_holiday'] = (full_test_df['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

# Assuming 5 days a week for all the given buildings
full_train_df.loc[(full_train_df['weekday'] == 5) | (full_train_df['weekday'] == 6) , 'is_holiday'] = 1
full_test_df.loc[(full_test_df['weekday']) == 5 | (full_test_df['weekday'] == 6) , 'is_holiday'] = 1

In [25]:
full_train_df = full_train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [26]:
full_test_df = full_test_df.drop(['timestamp'], axis=1)
full_train_df = full_train_df.drop(['timestamp'], axis=1)
print (f'Shape of training dataset: {full_train_df.shape}')
print (f'Shape of testing dataset: {full_test_df.shape}')

Shape of training dataset: (19869886, 80)
Shape of testing dataset: (41697600, 80)


In [27]:
## Reducing memory
full_train_df = reduce_mem_usage(full_train_df)
full_test_df = reduce_mem_usage(full_test_df)
gc.collect()

Mem. usage decreased to 3012.95 Mb (10.2% reduction)
Mem. usage decreased to 6402.31 Mb (10.1% reduction)


63

In [28]:
wind_cols = ['wind_direction','wind_direction_mean_lag3','wind_direction_max_lag3', 'wind_direction_min_lag3','wind_direction_std_lag3', 'wind_speed_mean_lag3',
'wind_speed_max_lag3', 'wind_speed_min_lag3', 'wind_speed_std_lag3']

In [29]:
full_train_df[wind_cols].isnull().sum()

wind_direction               0
wind_direction_mean_lag3     0
wind_direction_max_lag3      0
wind_direction_min_lag3      0
wind_direction_std_lag3     15
wind_speed_mean_lag3         0
wind_speed_max_lag3          0
wind_speed_min_lag3          0
wind_speed_std_lag3         15
dtype: int64

## Below is the hot 🥵 fix for the expense of time

In [30]:
full_train_df['wind_direction_std_lag3'].fillna(0,inplace=True)
full_train_df['wind_speed_std_lag3'].fillna(0,inplace=True)
full_test_df['wind_direction_std_lag3'].fillna(0,inplace=True)
full_test_df['wind_speed_std_lag3'].fillna(0,inplace=True)

In [31]:
for wcol in wind_cols:
    full_train_df[wcol] = full_train_df[wcol].apply(degToCompass)
    full_test_df[wcol] = full_test_df[wcol].apply(degToCompass)

In [32]:
beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

for item in beaufort:
    full_train_df.loc[(full_train_df['wind_speed']>=item[1]) & (full_train_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]
    full_test_df.loc[(full_test_df['wind_speed']>=item[1]) & (full_test_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

In [33]:
full_train_df = reduce_mem_usage(full_train_df)
full_test_df = reduce_mem_usage(full_test_df)
gc.collect()

Mem. usage decreased to 2880.31 Mb (31.2% reduction)
Mem. usage decreased to 6123.95 Mb (30.9% reduction)


14

## Export engineered data for model building

In [34]:
# Save the testing dataset to freeup the RAM. We'll read after training
full_test_df.to_pickle('tmp/full_test_df.pkl')
full_train_df.to_pickle('tmp/full_train_df.pkl')

In [35]:
==================

SyntaxError: invalid syntax (<ipython-input-35-0a58694ad6d2>, line 1)

In [ ]:
from featexp import get_trend_stats

In [ ]:
stats = get_trend_stats(data=full_train_df, target_col='meter_reading', data_test=full_test_df)

In [ ]:
import pandas as pd